In [62]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import imageio
import os 


## Preprocessing Images

In [74]:
## Train Positive
pos_train_path = "Data/chest_xray/train/PNEUMONIA/"
train_pos_images = os.listdir(pos_train_path)
resized_positive_save_path = "resizedImages/train/PNEUMONIA/"
for x in train_pos_images:
    img_path = pos_train_path+x
    img_path = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img_path, channels=3)
    resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.uint8)
    imageio.imwrite(resized_positive_save_path+x, resized_img)
    print("img {} done".format(x))    

print("******************** TRAIN POS COMPLETED *********************************")
## Train Negative

neg_train_path = "Data/chest_xray/train/NORMAL/"
train_neg_images = os.listdir(neg_train_path)
resized_negative_save_path = "resizedImages/train/NORMAL/"
for x in train_neg_images:
    img_path = neg_train_path+x
    img_path = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img_path, channels=3)
    resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.uint8)
    imageio.imwrite(resized_negative_save_path+x, resized_img)
    print("img {} done".format(x))    
print("******************** TRAIN NEG COMPLETED *********************************")

## Test Positive
pos_test_path = "Data/chest_xray/test/PNEUMONIA/"
test_pos_images = os.listdir(pos_test_path)
resized_positive_save_path = "resizedImages/test/PNEUMONIA/"
for x in test_pos_images:
    img_path = pos_test_path+x
    img_path = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img_path, channels=3)
    resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.uint8)
    imageio.imwrite(resized_positive_save_path+x, resized_img)
    print("img {} done".format(x))    

print("******************** TEST POS COMPLETED *********************************")

## Test Negative
neg_test_path = "Data/chest_xray/test/NORMAL/"
test_neg_images = os.listdir(neg_test_path)
resized_negative_save_path = "resizedImages/test/NORMAL/"
for x in test_neg_images:
    img_path = neg_test_path+x
    img_path = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img_path, channels=3)
    resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.uint8)
    imageio.imwrite(resized_negative_save_path+x, resized_img)
    print("img {} done".format(x))    
print("******************** TEST NEG COMPLETED *********************************")


        
        

img person63_bacteria_306.jpeg done
img person1438_bacteria_3721.jpeg done
img person755_bacteria_2659.jpeg done
img person478_virus_975.jpeg done
img person661_bacteria_2553.jpeg done
img person276_bacteria_1296.jpeg done
img person1214_bacteria_3166.jpeg done
img person1353_virus_2333.jpeg done
img person26_bacteria_122.jpeg done
img person124_virus_238.jpeg done
img person1619_bacteria_4261.jpeg done
img person890_bacteria_2814.jpeg done
img person1514_bacteria_3964.jpeg done
img person1491_bacteria_3893.jpeg done
img person69_bacteria_338.jpeg done
img person321_bacteria_1489.jpeg done
img person281_bacteria_1329.jpeg done
img person688_bacteria_2584.jpeg done
img person1598_bacteria_4197.jpeg done
img person535_virus_1062.jpeg done
img person1222_bacteria_3177.jpeg done
img person581_bacteria_2392.jpeg done
img person1233_virus_2090.jpeg done
img person1816_bacteria_4674.jpeg done
img person1812_bacteria_4667.jpeg done
img person1253_virus_2129.jpeg done
img person407_virus_811.jp

img person362_virus_741.jpeg done
img person1935_bacteria_4848.jpeg done
img person1593_virus_2767.jpeg done
img person1304_bacteria_3269.jpeg done
img person1381_virus_2375.jpeg done
img person1608_bacteria_4235.jpeg done
img person445_virus_916.jpeg done
img person302_bacteria_1430.jpeg done
img person1188_bacteria_3137.jpeg done
img person353_virus_728.jpeg done
img person348_bacteria_1602.jpeg done
img person1272_bacteria_3232.jpeg done
img person1294_virus_2222.jpeg done
img person3_bacteria_12.jpeg done
img person124_virus_234.jpeg done
img person378_virus_761.jpeg done
img person1478_bacteria_3848.jpeg done
img person1446_bacteria_3740.jpeg done
img person23_bacteria_91.jpeg done
img person294_bacteria_1381.jpeg done
img person462_bacteria_1967.jpeg done
img person1539_bacteria_4022.jpeg done
img person1839_bacteria_4705.jpeg done
img person299_bacteria_1417.jpeg done
img person1535_bacteria_4017.jpeg done
img person1507_bacteria_3945.jpeg done
img person1429_bacteria_3688.jpeg 

img person622_bacteria_2494.jpeg done
img person530_bacteria_2231.jpeg done
img person7_bacteria_29.jpeg done
img person684_virus_1275.jpeg done
img person326_bacteria_1507.jpeg done
img person260_virus_541.jpeg done
img person370_bacteria_1690.jpeg done
img person1142_virus_1892.jpeg done
img person1545_bacteria_4041.jpeg done
img person81_virus_152.jpeg done
img person1194_bacteria_3143.jpeg done
img person1465_virus_2537.jpeg done
img person317_virus_643.jpeg done
img person476_bacteria_2026.jpeg done
img person1537_bacteria_4019.jpeg done
img person1620_bacteria_4272.jpeg done
img person779_virus_1410.jpeg done
img person419_bacteria_1845.jpeg done
img person701_virus_1297.jpeg done
img person366_virus_746.jpeg done
img person7_bacteria_28.jpeg done
img person978_virus_1653.jpeg done
img person1737_bacteria_4573.jpeg done
img person1451_virus_2479.jpeg done
img person326_bacteria_1506.jpeg done
img person370_bacteria_1691.jpeg done
img person81_virus_153.jpeg done
img person1488_vi

img person1344_virus_2319.jpeg done
img person399_virus_793.jpeg done
img person1455_virus_2492.jpeg done
img person926_virus_1583.jpeg done
img person1275_bacteria_3237.jpeg done
img person837_bacteria_2753.jpeg done
img person514_bacteria_2184.jpeg done
img person900_virus_1554.jpeg done
img person348_bacteria_1603.jpeg done
img person995_virus_1676.jpeg done
img person1512_bacteria_3958.jpeg done
img person1138_bacteria_3080.jpeg done
img person340_virus_698.jpeg done
img person966_bacteria_2891.jpeg done
img person491_bacteria_2073.jpeg done
img person3_bacteria_13.jpeg done
img person1413_bacteria_3615.jpeg done
img person1310_bacteria_3304.jpeg done
img person1546_virus_2687.jpeg done
img person23_bacteria_90.jpeg done
img person294_bacteria_1380.jpeg done
img person671_bacteria_2564.jpeg done
img person281_bacteria_1324.jpeg done
img person325_bacteria_1497.jpeg done
img person321_bacteria_1484.jpeg done
img person498_virus_1007.jpeg done
img person415_virus_847.jpeg done
img pe

img person1936_bacteria_4852.jpeg done
img person847_virus_1492.jpeg done
img person59_bacteria_284.jpeg done
img person116_virus_221.jpeg done
img person354_virus_729.jpeg done
img person260_bacteria_1224.jpeg done
img person983_virus_1660.jpeg done
img person1491_bacteria_3892.jpeg done
img person1149_bacteria_3094.jpeg done
img person257_bacteria_1200.jpeg done
img person816_bacteria_2727.jpeg done
img person321_bacteria_1488.jpeg done
img person281_bacteria_1328.jpeg done
img person361_bacteria_1651.jpeg done
img person320_virus_647.jpeg done
img person266_bacteria_1242.jpeg done
img person581_bacteria_2393.jpeg done
img person489_bacteria_2063.jpeg done
img person1093_bacteria_3033.jpeg done
img person357_bacteria_1640.jpeg done
img person259_bacteria_1217.jpeg done
img person1012_bacteria_2943.jpeg done
img person1159_bacteria_3104.jpeg done
img person276_bacteria_1297.jpeg done
img person472_virus_969.jpeg done
img person1034_bacteria_2968.jpeg done
img person1250_bacteria_3207.

...

img IM-0634-0001.jpeg done
img NORMAL2-IM-1245-0001.jpeg done
img IM-0128-0001.jpeg done
img NORMAL2-IM-1073-0001.jpeg done
img NORMAL2-IM-0596-0001.jpeg done
img NORMAL2-IM-0419-0001.jpeg done
img NORMAL2-IM-0812-0001.jpeg done
img NORMAL2-IM-0509-0001.jpeg done
img NORMAL2-IM-0486-0001.jpeg done
img NORMAL2-IM-0954-0001.jpeg done
img IM-0724-0001.jpeg done
img NORMAL2-IM-1163-0001.jpeg done
img IM-0512-0001.jpeg done
img IM-0348-0001.jpeg done
img IM-0662-0001.jpeg done
img IM-0381-0001.jpeg done
img NORMAL2-IM-1025-0001.jpeg done
img NORMAL2-IM-0435-0001.jpeg done
img NORMAL2-IM-0603-0001.jpeg done
img NORMAL2-IM-1067-0001-0001.jpeg done
img NORMAL2-IM-0868-0001.jpeg done
img NORMAL2-IM-0573-0001.jpeg done
img IM-0332-0001.jpeg done
img IM-0618-0001.jpeg done
img NORMAL2-IM-1269-0001.jpeg done
img NORMAL2-IM-1096-0001.jpeg done
img IM-0274-0001.jpeg done
img IM-0568-0001.jpeg done
img IM-0618-0001-0001.jpeg done
img IM-0326-0001.jpeg done
img NORMAL2-IM-0798-0001.jpeg done
img NORMA

img IM-0497-0001-0002.jpeg done
img NORMAL2-IM-1271-0001.jpeg done
img NORMAL2-IM-1047-0001.jpeg done
img NORMAL2-IM-0661-0001.jpeg done
img IM-0644-0001-0002.jpeg done
img NORMAL2-IM-0511-0001.jpeg done
img NORMAL2-IM-0727-0001.jpeg done
img IM-0399-0001.jpeg done
img IM-0350-0001.jpeg done
img IM-0166-0001.jpeg done
img IM-0485-0001.jpeg done
img IM-0216-0001.jpeg done
img IM-0728-0001.jpeg done
img NORMAL2-IM-1266-0001-0002.jpeg done
img IM-0458-0001.jpeg done
img IM-0491-0001.jpeg done
img IM-0172-0001.jpeg done
img NORMAL2-IM-1094-0001-0001.jpeg done
img NORMAL2-IM-0396-0001.jpeg done
img NORMAL2-IM-0675-0001.jpeg done
img NORMAL2-IM-0443-0001.jpeg done
img NORMAL2-IM-1053-0001.jpeg done
img IM-0614-0001.jpeg done
img IM-0564-0001.jpeg done
img IM-0752-0001.jpeg done
img IM-0278-0001.jpeg done
img NORMAL2-IM-1323-0001.jpeg done
img IM-0620-0001-0002.jpeg done
img NORMAL2-IM-0922-0001.jpeg done
img NORMAL2-IM-0439-0001.jpeg done
img NORMAL2-IM-0780-0001.jpeg done
img NORMAL2-IM-074

img person153_bacteria_725.jpeg done
img person147_bacteria_711.jpeg done
img person88_bacteria_437.jpeg done
img person99_bacteria_474.jpeg done
img person175_bacteria_833.jpeg done
img person144_bacteria_690.jpeg done
img person49_virus_101.jpeg done
img person85_bacteria_419.jpeg done
img person1668_virus_2882.jpeg done
img person147_bacteria_707.jpeg done
img person48_virus_100.jpeg done
img person111_bacteria_536.jpeg done
img person133_bacteria_637.jpeg done
img person141_bacteria_681.jpeg done
img person172_bacteria_828.jpeg done
img person121_bacteria_576.jpeg done
img person157_bacteria_739.jpeg done
img person135_bacteria_647.jpeg done
img person1657_virus_2864.jpeg done
img person1620_virus_2807.jpeg done
img person152_bacteria_723.jpeg done
img person141_bacteria_678.jpeg done
img person82_bacteria_402.jpeg done
img person130_bacteria_626.jpeg done
img person139_bacteria_664.jpeg done
img person145_bacteria_696.jpeg done
img person154_bacteria_728.jpeg done
img person159_ba

img person102_bacteria_487.jpeg done
img person111_bacteria_533.jpeg done
img person1645_virus_2845.jpeg done
img person57_virus_113.jpeg done
img person85_bacteria_417.jpeg done
img person124_bacteria_590.jpeg done
img person75_virus_136.jpeg done
img person45_virus_95.jpeg done
img person95_bacteria_463.jpeg done
img person56_virus_112.jpeg done
img person1625_virus_2817.jpeg done
img person88_bacteria_439.jpeg done
img person1635_virus_2831.jpeg done
img person81_bacteria_396.jpeg done
img person130_bacteria_623.jpeg done
img person139_bacteria_661.jpeg done
img person1619_virus_2806.jpeg done
img person1643_virus_2843.jpeg done
img person96_bacteria_464.jpeg done
img person90_bacteria_443.jpeg done
img person114_bacteria_544.jpeg done
img person1685_virus_2903.jpeg done
img person150_bacteria_716.jpeg done
img person146_bacteria_704.jpeg done
img person113_bacteria_542.jpeg done
img person26_virus_60.jpeg done
img person29_virus_64.jpeg done
img person142_bacteria_683.jpeg done
img

img IM-0011-0001-0002.jpeg done
img IM-0070-0001.jpeg done
img NORMAL2-IM-0294-0001.jpeg done
img NORMAL2-IM-0340-0001.jpeg done
img NORMAL2-IM-0206-0001.jpeg done
img NORMAL2-IM-0030-0001.jpeg done
img IM-0103-0001.jpeg done
img IM-0045-0001.jpeg done
img NORMAL2-IM-0354-0001.jpeg done
img NORMAL2-IM-0041-0001.jpeg done
img NORMAL2-IM-0277-0001.jpeg done
img NORMAL2-IM-0107-0001.jpeg done
img NORMAL2-IM-0331-0001.jpeg done
img IM-0093-0001.jpeg done
img NORMAL2-IM-0219-0001.jpeg done
img IM-0087-0001.jpeg done
img NORMAL2-IM-0173-0001-0001.jpeg done
img NORMAL2-IM-0325-0001.jpeg done
img IM-0015-0001.jpeg done
img NORMAL2-IM-0238-0001.jpeg done
img NORMAL2-IM-0132-0001.jpeg done
img NORMAL2-IM-0304-0001.jpeg done
img NORMAL2-IM-0256-0001.jpeg done
img NORMAL2-IM-0060-0001.jpeg done
img NORMAL2-IM-0310-0001.jpeg done
img NORMAL2-IM-0195-0001.jpeg done
img IM-0001-0001.jpeg done
img IM-0111-0001.jpeg done
img NORMAL2-IM-0346-0001.jpeg done
img NORMAL2-IM-0352-0001.jpeg done
img IM-0039-

## Returning Img, Label pairs

In [86]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def returnLabel(path):
    imgClass = tf.strings.split(path,'/')[-2]
    return 0 if imgClass == 'NORMAL' else 1

def returnImg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img)
    imgClass = returnLabel(path)
    return img, imgClass

In [87]:
train_path = 'resizedImages/train'
list_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))

labeled_ds = list_ds.map(returnImg, num_parallel_calls=AUTOTUNE)

In [100]:
for x in labeled_ds.take(10):
    img, label = x
    print('label: {}, shape: {}'.format(label, img.shape))

label: 1, shape: (128, 128, 3)
label: 0, shape: (128, 128, 3)
label: 0, shape: (128, 128, 3)
label: 0, shape: (128, 128, 3)
label: 1, shape: (128, 128, 3)
label: 1, shape: (128, 128, 3)
label: 0, shape: (128, 128, 3)
label: 1, shape: (128, 128, 3)
label: 1, shape: (128, 128, 3)
label: 1, shape: (128, 128, 3)
